In [9]:
import numpy as np
import optuna
from sklearn import metrics
import warnings
import pickle
warnings.simplefilter(action='ignore', category=FutureWarning)
import joblib
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, r2_score, roc_curve
import matplotlib.pyplot as plt

In [10]:
# Load the data
data = np.load('Classified_Data_Var.npz')

# Convert the data to PyTorch tensors
X = torch.Tensor(data['features'])
y = torch.Tensor(data['labels']).long()

print(X.shape)  # Should output: (41740, 9, 9)
print(y.shape)  # Should output: (41740,)

# Split the data into training+validation set and test set
X_train_val, X_test, y_train_val, y_test = train_test_split(X.numpy(), y.numpy(), test_size=0.2, random_state=42)

# Further split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 of the train_val set, i.e., 20% of the original data

# If you need to flatten the 9x9 matrices into 81-element vectors, do this:
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Standardize the training and validation data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Convert the numpy arrays back to tensors
X_train = torch.Tensor(X_train)
X_val = torch.Tensor(X_val)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train).long()
y_val = torch.Tensor(y_val).long()
y_test = torch.Tensor(y_test).long()

torch.Size([41740, 18])
torch.Size([41740])


In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.head_dim = embed_size // num_heads

        assert (
            self.head_dim * num_heads == embed_size
        ), "Embed size needs to be divisible by number of heads"

        self.values = nn.Linear(embed_size, self.num_heads * self.head_dim, bias=False)
        self.keys = nn.Linear(embed_size, self.num_heads * self.head_dim, bias=False)
        self.queries = nn.Linear(embed_size, self.num_heads * self.head_dim, bias=False)
        self.fc_out = nn.Linear(num_heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Split the embedding into `num_heads` different pieces
        values = self.values(values).view(values.shape[0], values.shape[1], self.num_heads, self.head_dim)
        keys = self.keys(keys).view(keys.shape[0], keys.shape[1], self.num_heads, self.head_dim)
        queries = self.queries(query).view(query.shape[0], query.shape[1], self.num_heads, self.head_dim)

        # Transpose for the dot product matrix multiplication to be computable
        values = values.transpose(1, 2)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)

        # Compute the attention scores
        attention = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            attention = attention.masked_fill(mask == 0, float("-1e20"))

        attention = F.softmax(attention / (self.embed_size ** (1 / 2)), dim=3)
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values])
        out = out.transpose(1, 2).contiguous().view(values.shape[0], -1, self.num_heads * self.head_dim)

        out = self.fc_out(out)
        return out

In [4]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        # Example feed-forward network
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, 2048),
            nn.ReLU(),
            nn.Linear(2048, embed_size)
        )

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        
        x = self.norm1(attention + query)
        forward = self.feed_forward(x)
        out = self.norm2(forward + x)
        return out

In [5]:
class Net(nn.Module):
    def __init__(self, embed_size, num_heads, num_classes):
        super(Net, self).__init__()
        self.transformer_block = TransformerBlock(embed_size, num_heads)
        
        self.fc1 = nn.Linear(embed_size * 2, 1024)  # Assuming a flatten operation before this layer
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 240)
        self.fc4 = nn.Linear(240, 168)
        self.fc_output = nn.Linear(168, num_classes)  # 最终输出类别数量


        self.bn = nn.BatchNorm1d(240)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x, mask=None):
        N = x.shape[0]
        x = x.view(N, 2, 9)  # Reshape to (batch_size, sequence_length, embed_size)
        # Apply transformer block
        x = self.transformer_block(x, x, x, mask)
        # Flatten the output for the fully connected layers
        x = x.view(N, -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.dropout(F.relu(self.bn(self.fc3(x))))
        x = F.relu(self.fc4(x))
        x = self.fc_output(x)
        return F.log_softmax(x, dim=1)
    

embed_size = 9  # Size of each input token's embedding
num_heads = 3  # Number of heads in the multi-head attention mechanism
num_classes = 2  # Number of output classes



In [8]:
# 检查CUDA是否可用并设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = Net(embed_size, num_heads, num_classes)
# 将模型移到设备上
model = net.to(device)

# 损失函数和优化器
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

# 学习率调度器
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.1, patience=10)

# 训练模型
def train_model(model, criterion, optimizer, scheduler, X_train, y_train, X_val, y_val, n_epochs, device):
    # 确保输入数据在正确的设备上
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    X_val = X_val.to(device)
    y_val = y_val.to(device)
    
    for epoch in range(n_epochs):
        model.train()  # 设置模型为训练模式
        optimizer.zero_grad()  # 清零梯度
        
        # 前向传播
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()
        
        # 验证模型性能
        model.eval()  # 设置模型为评估模式
        with torch.no_grad():  # 不计算梯度，节省内存和计算资源
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)
            _, predicted = torch.max(val_outputs, 1)
            val_accuracy = (predicted == y_val).float().mean()
        
        # 更新学习率
        # scheduler.step(val_loss)  # 这里使用验证损失作为参数
        
        # 每隔一定epoch后，打印训练状态
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}, '
                  f'Validation Loss: {val_loss.item():.4f}, '
                  f'Validation Accuracy: {val_accuracy.item():.4f}')

    # 返回训练好的模型
    return model

# 注意：在实际使用之前，你需要定义X_train, y_train, X_val, y_val，并确保它们是正确的形状和类型，并且已经转移到了GPU
# X_train, y_train, X_val, y_val = ...

# 训练模型
trained_model = train_model(model, criterion, optimizer, None, X_train, y_train, X_val, y_val, 20000, device)

Epoch [10/20000], Loss: 0.6198, Validation Loss: 0.7244, Validation Accuracy: 0.6554
Epoch [20/20000], Loss: 0.6114, Validation Loss: 0.6206, Validation Accuracy: 0.6641
Epoch [30/20000], Loss: 0.6076, Validation Loss: 0.6123, Validation Accuracy: 0.6681
Epoch [40/20000], Loss: 0.6043, Validation Loss: 0.6135, Validation Accuracy: 0.6640
Epoch [50/20000], Loss: 0.6016, Validation Loss: 0.6131, Validation Accuracy: 0.6681
Epoch [60/20000], Loss: 0.5979, Validation Loss: 0.6140, Validation Accuracy: 0.6670
Epoch [70/20000], Loss: 0.5978, Validation Loss: 0.6110, Validation Accuracy: 0.6700
Epoch [80/20000], Loss: 0.5900, Validation Loss: 0.6100, Validation Accuracy: 0.6723
Epoch [90/20000], Loss: 0.5874, Validation Loss: 0.6095, Validation Accuracy: 0.6688
Epoch [100/20000], Loss: 0.5811, Validation Loss: 0.6098, Validation Accuracy: 0.6691
Epoch [110/20000], Loss: 0.5843, Validation Loss: 0.6100, Validation Accuracy: 0.6699
Epoch [120/20000], Loss: 0.5746, Validation Loss: 0.6125, Valid

KeyboardInterrupt: 

In [ ]:
# 创建数据加载器
train_dataset = TensorDataset(X_train, y_train)
valid_dataset = TensorDataset(X_val, y_val)  # 验证集
test_dataset = TensorDataset(X_test, y_test)  # 测试集

batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)  # 创建验证集的加载器
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)  # 测试集的加载器

In [ ]:
class Net(nn.Module):
    def __init__(self, trial):
        super(Net, self).__init__()
        # 使用 trial.suggest_int 来调整网络层的数量
        n_layers = trial.suggest_int('n_layers', 1, 10)
        layers = []

        in_features = 81 # 修改输入特征数量为 9x9 = 81
        for i in range(n_layers):
            out_features = trial.suggest_int('n_units_l{}'.format(i), 4, 128)
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            in_features = out_features

        layers.append(nn.Linear(in_features, 10)) # 假设有 10 个类别
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [ ]:
def objective(trial):
    # 为这次 trial 创建网络
    model = Net(trial).cuda()

    # 提议超参数
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)

    criterion = nn.CrossEntropyLoss()

    # 训练网络
    for epoch in range(10):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    # 验证网络
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / len(valid_loader.dataset)

    # Optuna 期望目标函数返回一个可以最小化或最大化的值
    return accuracy

In [ ]:
# 创建一个 study 对象并运行优化
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 最佳参数
print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial


[I 2024-03-26 13:21:42,892] A new study created in memory with name: no-name-3464095b-92a4-4924-a931-93c6b678e86e
[I 2024-03-26 13:21:47,984] Trial 0 finished with value: 0.1813608049832295 and parameters: {'n_layers': 3, 'n_units_l0': 71, 'n_units_l1': 118, 'n_units_l2': 72, 'optimizer': 'SGD', 'lr': 0.035010366300681724}. Best is trial 0 with value: 0.1813608049832295.
[I 2024-03-26 13:21:55,788] Trial 1 finished with value: 0.12038811691423096 and parameters: {'n_layers': 5, 'n_units_l0': 63, 'n_units_l1': 104, 'n_units_l2': 41, 'n_units_l3': 128, 'n_units_l4': 47, 'optimizer': 'Adam', 'lr': 0.08215233286760888}. Best is trial 0 with value: 0.1813608049832295.
[I 2024-03-26 13:22:03,612] Trial 2 finished with value: 0.1750119789171059 and parameters: {'n_layers': 5, 'n_units_l0': 4, 'n_units_l1': 65, 'n_units_l2': 112, 'n_units_l3': 48, 'n_units_l4': 99, 'optimizer': 'Adam', 'lr': 0.0002505123586834996}. Best is trial 0 with value: 0.1813608049832295.
[I 2024-03-26 13:22:10,643] Tri

KeyboardInterrupt: 

In [ ]:
print('Value: {}'.format(trial.value))
print('Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Value: 0.1889075227599425
Params: 
    n_layers: 4
    n_units_l0: 46
    n_units_l1: 88
    n_units_l2: 26
    n_units_l3: 73
    optimizer: Adam
    lr: 0.000951897561635773


In [ ]:
# 将study对象保存到文件
with open('Multiclass_Study.pkl', 'wb') as f:
    pickle.dump(study, f)

# 稍后时间点，从文件加载study对象
with open('Multiclass_Study.pkl', 'rb') as f:
    loaded_study = pickle.load(f)

In [ ]:
# 保存最佳参数到文件
joblib.dump(trial.params, 'best_params.pkl')

# 需要时加载最佳参数
best_params = joblib.load('best_params.pkl')

In [ ]:
# 创建最终模型使用最佳超参数
best_trial = study.best_trial
final_model = Net(trial).cuda()
NUM_EPOCHS = 300
# 选择最佳试验中推荐的优化器并设置学习率
optimizer_name = best_params['optimizer']
lr = best_params['lr']
optimizer = getattr(torch.optim, optimizer_name)(final_model.parameters(), lr=lr)

# 如果需要，您可以重新定义损失函数和数据加载器
criterion = nn.CrossEntropyLoss()
# train_loader = DataLoader(...)  # 假设您已经有了完整的训练数据加载器

# 训练最终模型
for epoch in range(NUM_EPOCHS):  # 设置您想要的训练周期数
    final_model.train()
    train_loss = 0
    # 使用tqdm封装您的数据加载器
    train_loader_tqdm = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS}', unit='batch')
    for batch_idx, (data, target) in enumerate(train_loader_tqdm):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = final_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # 设置进度条的后缀显示当前平均损失
        train_loader_tqdm.set_postfix(loss=train_loss/(batch_idx+1))


    # 每个周期结束后可以验证模型性能，并且可以选择保存性能最好的模型

# 可以选择保存模型
torch.save(final_model.state_dict(), 'final_model.pth')

Epoch 1/300:   0%|          | 0/392 [00:00<?, ?batch/s]

Epoch 300/300: 100%|██████████| 392/392 [00:01<00:00, 361.70batch/s, loss=1.78]


In [ ]:
# 确保模型处于评估模式
final_model.eval()

# 准备数据结构来保存真实标签和预测值
true_labels = []
predicted_labels = []
predicted_probs = []

# 不计算梯度，以加快计算速度并减少内存消耗
with torch.no_grad():
    for data, target in test_loader:  # 假设你有一个test_loader
        data, target = data.cuda(), target.cuda()
        output = final_model(data)

        # 获得预测概率用于ROC AUC等计算
        probs = torch.softmax(output, dim=1)
        predicted_probs.append(probs.cpu().numpy())

        # 获取预测结果
        _, preds = torch.max(output, 1)
        true_labels.extend(target.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

# 将列表转换为NumPy数组
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)
predicted_probs = np.vstack(predicted_probs)

# 计算准确率
accuracy = accuracy_score(true_labels, predicted_labels)

# 计算宏平均召回率
recall = recall_score(true_labels, predicted_labels, average='macro')

# 计算ROC AUC（多分类需要处理）
# 假设是二分类问题，使用标签为1的正类概率
if predicted_probs.shape[1] == 2:
    roc_auc = roc_auc_score(true_labels, predicted_probs[:, 1])
else:
    # 使用one-vs-rest approach处理多分类问题的ROC AUC
    roc_auc = roc_auc_score(true_labels, predicted_probs, multi_class='ovr')

# 计算R^2得分
r2 = r2_score(true_labels, predicted_labels)

# 打印所有指标
print('Accuracy:', accuracy)
print('Recall:', recall)
print('ROC AUC:', roc_auc)
print('R^2 Score:', r2)

Accuracy: 0.17081935793004313
Recall: 0.16306448243243982
ROC AUC: 0.6119960344894373
R^2 Score: -0.39241241604133204


In [15]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 假设你的数据集和标签已经准备好，分别存储在 X 和 y 中
# X 的维度是 (41740, 9)，y 的维度是 (41740,)
X = np.random.rand(41740, 9)  # 这里用随机数据代替实际数据
y = np.random.randint(0, 2, 41740)  # 这里用随机标签代替实际标签
data = np.load('Classified_Data.npz')

X = data['features']
y = data['labels']
# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [16]:
# 创建决策树分类器实例
# 由于scikit-learn没有直接实现C4.5，我们可以通过设置criterion='entropy'来使用信息增益比
# 这是C4.5算法中用于选择特征的方法
classifier = DecisionTreeClassifier(criterion='entropy', random_state=42)

# 训练分类器
classifier.fit(X_train, y_train)

# 使用分类器进行预测
y_pred = classifier.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6158361284139914
